# Political Chatbot Analysis: Canadian Bill Importance Evaluation

This notebook performs an analysis of Canadian bills using a **text-based** approach. The goal is to evaluate the importance of each bill using **LLMs** and divide the text into smaller chunks to assess their significance in deciding whether to pass the bill.

In [ ]:
import json
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
import openai
from dotenv import load_dotenv
import os
from langchain.output_parsers import EnumOutputParser
from enum import Enum

load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
openai.api_key = openai_api_key

class ImportanceLevel(str, Enum):
    LOW = "LOW"
    MEDIUM = "MEDIUM"
    HIGH = "HIGH"

parser = EnumOutputParser(enum=ImportanceLevel)

with open("canadian-legislation-dataset/detailed_bills_with_full_text.json", "r", encoding="utf-8") as file:
    bills_data = json.load(file)

full_text = bills_data[0]['full_text'] 

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,  
    length_function=len,  
    is_separator_regex=False,  
)

chunks = text_splitter.split_text(full_text)

llm = ChatOpenAI(model="gpt-4o-mini", openai_api_key=openai_api_key, temperature=0.2)

prompt_template = """
You are a legislative assistant tasked with analyzing sections of a bill. Your goal is to assess the importance of the given section in deciding whether the bill should be passed.

Here is the section of the bill:
{text}

Classify the importance as either LOW, MEDIUM, or HIGH (respond with only one of these words):
"""

prompt = PromptTemplate(input_variables=["text"], template=prompt_template)

def analyze_importance(chunk):
    formatted_prompt = prompt.format(text=chunk)
    result = llm.invoke(formatted_prompt).content
    importance = parser.parse(result.strip())
    return importance

importances = [analyze_importance(chunk) for chunk in chunks]

for i, (chunk, importance) in enumerate(zip(chunks, importances)):
    print(f"\nChunk {i+1}:")
    print(f"Text preview: {chunk[:100]}...")
    print(f"Importance: {importance}")